In this short programming assignment we will look into applications of word embeddings in similarity search and nearest neighbors. We will also look at creating a video based on tSNE embeddings of images.

In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import csv

from sklearn.manifold import TSNE

## -1. Create a video from images
Download any 1000 or more 'appropriate' and publicly available images from the web. This could be part of a data set or something specific that you picked up or are interested in.

We discussed using tSNE to find image embeddings for these images. Apply the tSNE library to these images and construct low-dimensional embeddings for the images. Use these embeddings to then:

a) Start at any random image in the data set 

b) Sequentially chain the next image to the previous image using a scoring function/probability based on the tSNE embedding. So you want to chain the most similar image to the current one and so on. Choose a frame rate that is appropriate to convert this chain of images into a video. Your video shouldn't be more than 3 minutes long.

c) Upload this video to youtube and share a link with your submission. 

d) Feel free to share your video to Discord to see what cool videos we come up with!

In [3]:
flow_path = "/Users/qingchuanhou/Desktop/flower_train/"
n = 1000

def Img_list(path):
    X = list()

    for label in os.listdir(path):
        if not label.startswith('.'):
            label_path = os.path.join(path, label)

            i = 0
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image_data = Img2data(image_path)
                X += [image_data]
                i += 1
                if i > n/5 - 1:
                    break
    return np.array(X)      
    


def Img2data(image_path):
    image = cv2.imread(image_path)
    image_resize = cv2.resize(image, (224, 224))
    image_data = np.reshape(image_resize, 224*224*3)
    return image_data

def Data2img(image_data):
    image = np.reshape(image_data, (224,224,3))
    return image

flowr_list = Img_list(flow_path)
np.shape(flowr_list)


(1000, 150528)

In [4]:
# TSNE in 1d
tsne_data = TSNE(n_components=1, learning_rate='auto', init='random').fit_transform(flowr_list)


In [5]:
image_sort = [x for _,x in sorted(zip(tsne_data, flowr_list))]

In [6]:
def video_out(images):
    out = cv2.VideoWriter('tsne_image.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, (224,224))

    for i in range(len(images)):
        out.write(Data2img(images[i]))

    out.release()

# video_out(image_sort)

In [7]:
# tSNE in 2d
tsne_data2 = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(flowr_list)


In [8]:
def sortby2d(img_list, tsne_data2d):
    out2 = cv2.VideoWriter('tsne_image2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, (224,224))

    x = tsne_data2d[0][0]
    y = tsne_data2d[0][1]
    tsne_data2d = np.delete(tsne_data2d, 0, 0)
    out.write(Data2img(img_list[0]))
    img_list = np.delete(img_list, 0, 0)
    for a in range(len(img_list)):
        min_dis = ((x - tsne_data2d[0][0])**2 + (y - tsne_data2d[0][1])**2)**0.5
        m = 0
        for i in range(len(tsne_data2d)):
            if min(min_dis, ((x - tsne_data2d[i][0])**2 + (y - tsne_data2d[i][1])**2)**0.5) < min_dis:
                min_dis = ((x - tsne_data2d[i][0])**2 + (y - tsne_data2d[i][1])**2)**0.5
                m = i
        x = tsne_data2d[m][0]
        y = tsne_data2d[m][1]
        tsne_data2d = np.delete(tsne_data2d, m, 0)
        out2.write(Data2img(img_list[m]))
        img_list = np.delete(img_list, m, 0)
    out2.release()
    
# sortby2d(flowr_list, tsne_data2)

Video using 1d tSNE: https://youtu.be/qEiuoSa4KfU

Video using 2d tSNE: https://youtu.be/EAUB0SLwrzQ

## Diving into Cheat Sheet of Pandas Data Frame
There are some useful functions for solving problems below when it comes to index and slice data frames. Let's go over them.
More materials can be found here: https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf
1. DataFrame() Construct a dataframe. Use it for putting a numpy ndarray into a dataframe.
1. DataFrame.loc() Purely label-location based indexer for selection by label. Use it for selecting word vectors in the dataframe.     
1. DataFrame.dot() Matrix multiplication with DataFrame. Use it for dot product of word vectors.
1. DataFrame.sort_values() Sort by the values along either axis. Use it for sorting distance short to long.

Below are examples of using these functions. You don't have to code anything in this block, just focus on understanding the functions and how it works in pandas.

In [9]:
# Define a ndarray
d = np.array([[0.1,0.3,0.4,0.5],[0.3,0.4,0.9,0.5],[0.2,0.8,0.7,0.5]], dtype=float, order='F')
print("Define sample word vectors")
print(d)
#Construct a dataframe from ndarray and index each row as word vectors
df = pd.DataFrame(d,index = ['word1','word2','word3'])
print("\nPandas Data Frame for word vectors")
print(df)
#Select word vector1 by its label
print("\nFind the row corresponding to word1")
print(df.loc['word1'])
#Calculate dot product of word vector1 and word vector2
print("\nCalculate the dot product between word1 and word2")
dot_product = df.loc['word2'].dot(df.loc['word1'])
print(dot_product)
#Calculate dot product of word vector1 to the rest of words
print("\nCalculate the dot product between word1 and rest of the words")
words_rest = ['word2','word3']
dot_product2 = df.loc[words_rest].dot(df.loc['word1'])
print(dot_product2)
#Sort Values of dot_product2 by high to low
print("\nSorted dot product values")
print(dot_product2.sort_values(ascending = False))

Define sample word vectors
[[0.1 0.3 0.4 0.5]
 [0.3 0.4 0.9 0.5]
 [0.2 0.8 0.7 0.5]]

Pandas Data Frame for word vectors
         0    1    2    3
word1  0.1  0.3  0.4  0.5
word2  0.3  0.4  0.9  0.5
word3  0.2  0.8  0.7  0.5

Find the row corresponding to word1
0    0.1
1    0.3
2    0.4
3    0.5
Name: word1, dtype: float64

Calculate the dot product between word1 and word2
0.76

Calculate the dot product between word1 and rest of the words
word2    0.76
word3    0.79
dtype: float64

Sorted dot product values
word3    0.79
word2    0.76
dtype: float64


## Dataset Details - Standford's GloVe pre-trained word vectors

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus. The GloVe pre-trained word vectors dataset contains English word vectors pre-trained on the combined Wikipedia 2014 + Gigaword 5th Edition corpora (6B tokens, 400K vocab). All tokens are in lowercase. This dataset contains 50-dimensional, 100-dimensional and 200-dimensional pre trained word vectors. In this problem we are going to use the 50-dimensional dataset. 

## \# 0. Get an overview on what Glove is
Read up the documentation on glove embeddings, esp. where it gets applied here: https://nlp.stanford.edu/projects/glove/

## Load Dataset
Let's load the dataset first. Each row is indexed as a word vector. Dimension of word vectors is 50. How many words are there in this dataset? Print a few words and see what they are. You don't need to code anything here, just understand the data structure.

In [10]:
# Load GloVe pre-trained vectors 
local_file1="/Users/qingchuanhou/Desktop/glove/glove.6B.50d.txt" # Make sure this file exists!
data = csv.reader
df = pd.read_csv(local_file1,sep=' ',index_col=0,header=None,engine='python',error_bad_lines=False, quoting=csv.QUOTE_NONE)
print("dataset shape - Rows: %d, Cols: %d" % (df.shape[0], df.shape[1]))
words = list(df.index)
print("print a few words in the dataset:", words[30:40])

/Users/qingchuanhou/opt/anaconda3/envs/Master/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


dataset shape - Rows: 400000, Cols: 50
print a few words in the dataset: ['be', 'has', 'are', 'have', 'but', 'were', 'not', 'this', 'who', 'they']


## \# 1. Print the first few 11 rows of the pandas data frame below

In [11]:
# Your code HERE - It should execute as expected! 
# (Search for a pandas functionality that can help you do this!)
df.head(11)

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
the,0.418000,0.249680,-0.41242,0.121700,0.345270,-0.044457,-0.49688,-0.178620,-0.000660,-0.656600,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.442510,0.187850,0.002785,-0.184110,-0.115140,-0.785810
",",0.013441,0.236820,-0.16899,0.409510,0.638120,0.477090,-0.42852,-0.556410,-0.364000,-0.239380,...,-0.080262,0.630030,0.321110,-0.467650,0.227860,0.360340,-0.378180,-0.566570,0.044691,0.303920
.,0.151640,0.301770,-0.16763,0.176840,0.317190,0.339730,-0.43478,-0.310860,-0.449990,-0.294860,...,-0.000064,0.068987,0.087939,-0.102850,-0.139310,0.223140,-0.080803,-0.356520,0.016413,0.102160
of,0.708530,0.570880,-0.47160,0.180480,0.544490,0.726030,0.18157,-0.523930,0.103810,-0.175660,...,-0.347270,0.284830,0.075693,-0.062178,-0.389880,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
to,0.680470,-0.039263,0.30186,-0.177920,0.429620,0.032246,-0.41376,0.132280,-0.298470,-0.085253,...,-0.094375,0.018324,0.210480,-0.030880,-0.197220,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
and,0.268180,0.143460,-0.27877,0.016257,0.113840,0.699230,-0.51332,-0.473680,-0.330750,-0.138340,...,-0.069043,0.368850,0.251680,-0.245170,0.253810,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
in,0.330420,0.249950,-0.60874,0.109230,0.036372,0.151000,-0.55083,-0.074239,-0.092307,-0.328210,...,-0.486090,-0.008027,0.031184,-0.365760,-0.426990,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
a,0.217050,0.465150,-0.46757,0.100820,1.013500,0.748450,-0.53104,-0.262560,0.168120,0.131820,...,0.138130,0.369730,-0.642890,0.024142,-0.039315,-0.260370,0.120170,-0.043782,0.410130,0.179600
"""",0.257690,0.456290,-0.76974,-0.376790,0.592720,-0.063527,0.20545,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650


## \# 2. Words Similarity

Similar words have similar embeddings (or vector values). We can use cosine similarity i.e. cos(u,v) = u.v/(|u||v|) to measure vector similarity. u.v is dot product of vectors, |u| is L2 norm of u. Remember, we spoke about computing similarity based on cosine-similarity (as another alternative to correlation) in class?

1. Normalize matrix df by norm of word vectors. 
1. Define a function to find words similarity to a given word.
1. Use the function defined to find the word in examples that is most similar to "happy".

In [12]:
vecter_facter = list(np.sqrt(np.square(df).sum(axis=1)))


In [13]:
vector_norm.head(10)

NameError: name 'vector_norm' is not defined

In [ ]:
## YOUR CODE HERE
# 1a. Calculate norm of word vectors
# What would be the dimension of the vector_norm array?
vector_norm = np.sqrt(np.square(df).sum(axis=1))

# 1b. Normalize matrix df by norm using .div()
dfn = 

# 2. Define a function to find words similar to a given word in a normalized dataframe
def find_word_similarity(word, examples,dataframe):
    # Input: word - one string
    #        examples - List of strings
    #        dataframe - An indexed normalized dataframe
    ## YOUR CODE HERE
    # Calculate dot product of each word in examples to the given word, sorted by value high to low
    # Once you have the sorted values of dot products (notice because of normalization, the dot product is the cosine similarity!),
    # obtain the words corresponding to the sorted values and call it similar_words
    similar_words = 

    # Return words similar to the given word
    return similar_words
    
examples = ["sad", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]

# 3.
# Use above function to calculate examples' similarity to happy (both "happy" and words in examples are in dfn)
print find_word_similarity("happy", examples,dfn)

SyntaxError: invalid syntax (3803129606.py, line 7)

In sklean library,there is a cosine_similarity fuction that directly calcualtes vectors similarity (you don't need to normalize vectors first). Let's use this function to calculate similarity again to confirm we get same results. 
For more information, please see here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
examples = ["sad", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity2 = pd.DataFrame(cosine_similarity(np.array(df.loc['happy']).reshape(1,50),np.array(df.loc[examples])),columns = examples)
print(similarity2.T)

## \# 3. Goodness of similarity
Comment on the how good the glove embeddings are on finding similar words to a given word using cosine similarity? Glove and word2vec embeddings are based on co-occurence of words in senetences across hundreds of thousands of documents on the web. Would this help explain your observations on word similarity?
What if you replace happy with sad, how do the results change?

## \# 4. Correlations
(This question is more of a reflection and building your intuition on how correlations we spoke in class connects to a real-world data set -  Open ended!)
What are some of the most correlated words from the similarity search you did earlier to the word "happy" and "sad". Likewise, what are some of the most uncorrelated words to "happy" and "sad". Does this make sense? How would you improve the results ? If "happy" were a random variable and "sad" was a random variable - What factors make the correlation between "happy" and "sad" (as you computed above) high?

## \# 5. Find nearest neighbourhood

It is helpful to compute the nearest neighbors to a word based on the cosine similarity that we defined earlier, so that given a word we can compute which are the other words which are most similar to it. Sometimes, the nearest neighbors according to this metric reveal overlap of concepts or topics that a word shares. E.g. government might be related to the word politics because they both share topics related to public policy, politicians, parties, elections, etc. The idea is whatever embeddings we are using - word2vec or glove is "hopefully" able to capture these correlations right!

1. Define a function to find the top n similar words to a given word. You can use either dot product of vectors or cosine_simialrity function. Note the search space for words is coming from your pandas data frame (so unlike the similarity problem we worked on earlier, we are not restricted to only a few words to search from - the search space here is the entire vocab captured in your data frame).
1. Find 20 nearest neighborhood for words 'duck' and 'animal'.
1. Find neighborhood intersection of 'duck' and 'animal', to find which words are similar to both 'duck' and 'animal'. This is also related to a similarity measure called "Jaccard Similarity" - Read up on this here: https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
# define a function to find the top n similar words to a given word in the 'df'

# PART 1
def find_most_similar(df, word, n):
    # INPUT: 
    # df: Given Data frame
    # word: String
    # n: Number of similar words to return
    
    # OUTPUT:
    # the list of similar words to return
    
    ## YOUR CODE HERE
    # define and compute the most similar words
    # Use a similarity measure like cosine similarity (like earlier) to do so
    
    similar_words = 
    
    return similar_words


# PART 2
# find top 20 similar words to duck
simil1 = find_most_similar(df, "duck", 50)
# find top 20 similar words to animal
simil2 = find_most_similar(df, "animal", 50)

# PART 3
# find the intersection of simil1 and simil2
#intersection =  (concat function of pandas is helpful here)
print intersection

word_labels = ['duck', 'animal'] + list(intersection.index)



## \# 6 Word analogies

Suppose you know the word vectors for King, Man and Woman. What is your intuitive answer for the 'riddle', King - Man + Woman = ? 
Let's go through below steps to derive the answer for this 'riddle' using the word embeddings.

1. Use vector arithmetic to define a new vector which equals to k - m + w (e.g. king, man and woman combination).
2. Calculate similarity of all the words in the corpus to the new vector and sort them by their similarity high to low. 
3. Return the top n vectors which have the highest similarity to the new vector.
1. Find the answers for the riddles, 
    1. good:bad::up:?
    1. germany:merkel::america:?

In [ ]:
# define a function to solve the problem of x is to y as a is to ?
# 'n' is the number of top words similar to the vector to return
# 'dataframe' is the indexed dataframe that contains all the words

# PART 1,2,3 above (Fill in the missing pieces)
def solve_riddle(x, y, a, n, dataframe):
    ## YOUR CODE HERE
    # calculate the vector of a + y - x, where a, x, y are in dataframe
    cal_vec = 
    
    # calculate distance of words in dataframe to cal_vec, sorted by similarity high to low
    similarity = 

    # return top n words and distance that closest to cal_vec
    return similarity 

# Call the solve_riddle function to compute the top answers
print solve_riddle("man", "woman", "king", 5,df)

## YOUR CODE HERE
# Solve the other two riddles
# good:bad::up:?
# PART 4
print solve_riddle()

# germany:merkel::america:?
print solve_riddle()